In [1]:
from Parameters import ModelParams
from Model_Data_Loader import ModelDataLoader
from Star_Parameters_Class import StarParams
from Mass_Loss_Class import MassLoss
from Fractionation_Class import Fractionation
from Misc_Class import Misc

import numpy as np
import pandas as pd
import json

In [2]:
# ---------- Initialize parameters and classes ----------
params = ModelParams()
mearth = params.mearth
rearth = params.rearth
mass_loss = MassLoss(params)
fractionation = Fractionation(params)
star_params = StarParams(params)
misc = Misc(params)
loader = ModelDataLoader('/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/', params)

In [5]:
# ---------- Load data ----------
### option 1
# mass, radius, Teq = loader.load_single_ddat_file('3HHe_10H2O_subNeptune.ddat')

### option 2
with open("planet_params.json", "r") as file:
    planet_params = json.load(file)
selected_planet = "TRAPPIST-1 e"                     # change the planet here

if selected_planet in planet_params:
    new_FEUV = planet_params[selected_planet]["FEUV"]
    params.update_param("FEUV", new_FEUV)
    print(f"Updated FEUV for {selected_planet}: {params.get_param('FEUV'):.2e} erg cm-2 s-1")
else:
    print(f"Make sure the planet you selected exists in the list! Check the spelling.")

radius = np.array([planet_params[selected_planet]["radius"] * rearth]) # cm
mass = np.array([planet_params[selected_planet]["mass"] * mearth]) # g
Teq = np.array([planet_params[selected_planet]["teq"]]) # K

# ---------- Find unique temperatures ----------
unique_Teqs = np.unique(Teq)
print("Unique Teq values found:", unique_Teqs)

# ---------- Range of EUV flux values ----------
all_flux_results = []

for Teq_val in unique_Teqs:
    star_params.update_param("Teq", Teq_val)
    Fbol = star_params.get_Fbol_from_Teq()
    # flux_range = star_params.get_FEUV_range_any_age()   # if option 1 from load data
    flux_range = [params.get_param('FEUV')]             # if option 2 from load data (.json file)

    mask = np.isclose(Teq, Teq_val)
    mass_group = mass[mask]
    radius_group = radius[mask]
    teq_group = Teq[mask]
    print(f"\nProcessing group with Teq = {Teq_val} K and Fbol = {Fbol}")

    # ---------- Execute ----------
    for flux in flux_range:
        print(f"\n Running model with FEUV = {flux} erg/cm^2/s")
        params.update_param('FEUV', flux)

        # ---------- Mass loss model ----------
        mass_loss_results = mass_loss.compute_mass_loss_parameters(mass_group, radius_group, teq_group, pureHHe=True)
        results_for_this_flux = {'FEUV': flux, 'Teq': Teq_val, 'Fbol': Fbol, 'end_results': mass_loss_results}
        # mdot = mass_loss_results[0]['Mdot']
        # print(f"Mdot =", mdot)
        
        # ---------- Fractionation model ----------
        fractionation_results = fractionation.execute_self_consistent_fractionation(mass_loss_results, mass_loss, misc, params)
        results_for_this_flux = {'FEUV': flux, 'Teq': Teq_val, 'Fbol': Fbol, 'end_results': fractionation_results}
        mdot = fractionation_results[0]['Mdot']
        print(f"Mdot =", mdot)
        
        all_flux_results.append(results_for_this_flux)

Updated FEUV for TRAPPIST-1 d: 7.88e+02 erg cm-2 s-1
Unique Teq values found: [286.2]

Processing group with Teq = 286.2 K and Fbol = 1630472.4652794236

 Running model with FEUV = 788.455 erg/cm^2/s
Converged in 2 iterations for planet 0.4 M_earth.
Final mmw_outflow = 1.0
Planet with mass=0.39 M_earth results: Mdot = 172117671.1370734
Planet with mass=0.39 M_earth results: phi_O = 7.551179888817771e-55, phi_H = 5.0008509634158904e-11, x_O = 6.039915961550551e-44

Mdot = 172117671.1370734


In [ ]:
# save data for plotting
rows = []

for flux_data in all_flux_results:
    for res in flux_data['end_results']:
        # include FEUV, Teq and Fbol from the outer dict
        record = {
            'FEUV':  flux_data['FEUV'],
            'Teq':   flux_data['Teq'],
            'Fbol':  flux_data['Fbol'],
        }
        record.update(res)
        rows.append(record)

df = pd.DataFrame(rows)
df.to_csv('/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/3HHe_10H2O_subNeptune.csv', index=False)